## Chapter 7 - Supervised Machine Learning (COVID ICU)

http://blood.stemcells.cam.ac.uk/single_cell_atlas#data

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report


In [2]:
dfx = pd.read_csv("../../datasets/single_cell_rna/nestorowa_corrected_log2_transformed_counts.txt", sep=' ',  )
dfx['annotation'] = dfx.index.str[:4]
# dfx = dfx[dfx["annotation"] != "LT.H"]
y = dfx["annotation"].values.ravel()
dfx.head()

,1110032F04Rik,1110059E24Rik,1300017J02Rik,1600014C10Rik,1700017B05Rik,1700019G17Rik,1700024P16Rik,1700026L06Rik,1700029J07Rik,1700030K09Rik,...,Zscan29,Zswim3,Zswim4,Zswim6,Zufsp,Zxdb,Zxdc,Zyx,Zzz3,annotation
HSPC_001,0.0,0.000000,0.0,0.000000,1.225439,0.0,0.0,0.0,0.000000,0.0,...,0.739099,0.000000,0.000000,0.0,0.000000,0.0,0.000000,5.386021,10.541571,HSPC
HSPC_002,0.0,0.000000,0.0,2.238601,2.238601,0.0,0.0,0.0,9.298240,0.0,...,2.238601,0.000000,2.238601,0.0,2.238601,0.0,0.000000,9.164370,3.603855,HSPC
HSPC_003,0.0,2.795189,0.0,0.000000,1.989360,0.0,0.0,0.0,0.000000,0.0,...,4.234474,7.378198,0.000000,0.0,0.000000,0.0,0.000000,9.143168,2.310576,HSPC
HSPC_004,0.0,1.326478,0.0,1.326478,2.005685,0.0,0.0,0.0,0.000000,0.0,...,2.005685,1.326478,3.094148,0.0,5.788797,0.0,5.577271,4.136426,2.813850,HSPC
HSPC_006,0.0,7.348663,0.0,4.946766,0.000000,0.0,0.0,0.0,1.941849,0.0,...,8.441787,1.941849,3.926937,0.0,0.000000,0.0,0.000000,7.752007,8.871921,HSPC


In [3]:
dfx.shape

(1645, 3992)

In [4]:
len(dfx)

1645

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(dfx.drop(columns = ["annotation"]))

In [12]:
from sklearn.decomposition import PCA

pca = PCA(n_components=900, svd_solver='full')
pca.fit(X_scaled)
data_pca = pca.fit_transform(X_scaled)

print(pca.explained_variance_ratio_.sum())
print(pca.explained_variance_ratio_[:5])

0.8545985153116179
[0.08083444 0.0235883  0.01405723 0.00996533 0.00821934]


In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_pca, y, test_size=0.33)

In [21]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

       HSPC       0.66      0.13      0.21       229
       LT.H       1.00      0.04      0.07        54
       Prog       0.52      1.00      0.68       260

avg / total       0.63      0.53      0.42       543



In [24]:
from sklearn.metrics import accuracy_score

for i in range(1,10):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    print("n =", i, "acc =", accuracy_score(y_test, y_pred))

n = 1 acc = 0.5524861878453039
n = 2 acc = 0.6077348066298343
n = 3 acc = 0.5395948434622467
n = 4 acc = 0.585635359116022
n = 5 acc = 0.5340699815837937
n = 6 acc = 0.569060773480663
n = 7 acc = 0.5285451197053407
n = 8 acc = 0.5524861878453039
n = 9 acc = 0.5193370165745856


In [36]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=1000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

       HSPC       0.71      0.94      0.81       229
       LT.H       0.00      0.00      0.00        54
       Prog       0.93      0.85      0.89       260

avg / total       0.74      0.81      0.77       543



C:\Users\Saleh Alkhalifa\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1134: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  precision = _prf_divide(tp_sum, pred_sum,


In [25]:
from sklearn.svm import SVC
svc = SVC(kernel="rbf")
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

       HSPC       0.62      0.97      0.76       229
       LT.H       0.00      0.00      0.00        54
       Prog       0.95      0.67      0.79       260

avg / total       0.72      0.73      0.70       543



C:\Users\Saleh Alkhalifa\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1134: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  precision = _prf_divide(tp_sum, pred_sum,


In [29]:
from sklearn.svm import SVC
svc = SVC(kernel="linear")
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

       HSPC       0.83      0.90      0.87       229
       LT.H       0.82      0.67      0.73        54
       Prog       0.93      0.90      0.92       260

avg / total       0.88      0.88      0.88       543



In [28]:
from sklearn.svm import SVC
svc = SVC(kernel="poly")
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

       HSPC       0.66      0.33      0.44       229
       LT.H       0.00      0.00      0.00        54
       Prog       0.60      0.98      0.74       260

avg / total       0.56      0.61      0.54       543



In [34]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(max_depth=4)
dtc.fit(X_train, y_train)
y_pred = dtc.predict(X_test)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

       HSPC       0.71      0.82      0.76       229
       LT.H       0.57      0.30      0.39        54
       Prog       0.86      0.83      0.84       260

avg / total       0.77      0.77      0.76       543



In [35]:
from sklearn.metrics import accuracy_score

for i in range(1,10):
    dtc = DecisionTreeClassifier(max_depth=i)
    dtc.fit(X_train, y_train)
    y_pred = dtc.predict(X_test)
    print("n =", i, "acc =", accuracy_score(y_test, y_pred))

n = 1 acc = 0.714548802946593
n = 2 acc = 0.7734806629834254
n = 3 acc = 0.8213627992633518
n = 4 acc = 0.7716390423572744
n = 5 acc = 0.7882136279926335
n = 6 acc = 0.7771639042357275
n = 7 acc = 0.7697974217311234
n = 8 acc = 0.7421731123388582
n = 9 acc = 0.7495395948434622


In [18]:
from sklearn.ensemble import AdaBoostClassifier
adab = AdaBoostClassifier(n_estimators=1000)
adab.fit(X_train, y_train)
y_pred = adab.predict(X_test)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

       HSPC       0.67      0.94      0.78       229
       LT.H       0.75      0.22      0.34        54
       Prog       0.94      0.73      0.82       260

avg / total       0.80      0.77      0.76       543



In [19]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(n_estimators=1000, learning_rate=0.4)
gbc.fit(X_train, y_train)
y_pred = gbc.predict(X_test)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

       HSPC       0.80      0.90      0.85       229
       LT.H       0.76      0.52      0.62        54
       Prog       0.91      0.87      0.89       260

avg / total       0.85      0.85      0.85       543



In [20]:
from xgboost import XGBClassifier
xgb = XGBClassifier(n_estimators=10000)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
print(classification_report(y_test, y_pred))

C:\Users\Saleh Alkhalifa\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:25:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
             precision    recall  f1-score   support

       HSPC       0.81      0.90      0.85       229
       LT.H       0.76      0.54      0.63        54
       Prog       0.92      0.89      0.91       260

avg / total       0.86      0.86      0.86       543



C:\Users\Saleh Alkhalifa\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
